In [2]:
# Importing necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import itertools  # <-- Tambahkan impor itertools

# Step 1: Load and preprocess the dataset
# Load dataset from CSV
file_path = 'heart.csv'
data = pd.read_csv(file_path)

# Separate features (X) and labels (y)
X = data.iloc[:, :-1].values  # Assuming last column is the target
y = data.iloc[:, -1].values  # Target column

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# Step 2: Define the Multi-Layer Perceptron (MLP) model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_layers, activation_fn):
        super(MLP, self).__init__()
        layers = []
        current_size = input_size

        # Add hidden layers
        for neurons in hidden_layers:
            layers.append(nn.Linear(current_size, neurons))
            layers.append(activation_fn())
            current_size = neurons

        # Add output layer (binary classification: 2 classes)
        layers.append(nn.Linear(current_size, 2))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# Step 3: Define the training function
def train_model(model, criterion, optimizer, X_train, y_train, X_test, y_test, epochs, batch_size):
    # Data loader for mini-batches
    train_loader = torch.utils.data.DataLoader(
        dataset=torch.utils.data.TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True
    )

    test_loader = torch.utils.data.DataLoader(
        dataset=torch.utils.data.TensorDataset(X_test, y_test), batch_size=batch_size, shuffle=False
    )

    train_losses = []
    test_accuracies = []

    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0

        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        train_losses.append(epoch_loss / len(train_loader))

        # Evaluate model on test set
        model.eval()
        all_preds, all_labels = [], []
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                preds = model(X_batch).argmax(dim=1)
                all_preds.extend(preds.numpy())
                all_labels.extend(y_batch.numpy())

        accuracy = accuracy_score(all_labels, all_preds)
        test_accuracies.append(accuracy)

    return train_losses, test_accuracies

# Step 4: Experiment with hyperparameters
# Define hyperparameters to test
hidden_layer_configs = [[8], [16, 8], [32, 16, 8]]
activation_fns = [nn.ReLU, nn.Sigmoid, nn.Tanh]
epoch_counts = [10, 25, 50]
learning_rates = [0.1, 0.01, 0.001]
batch_sizes = [16, 32, 64]

results = []
input_size = X_train.shape[1]

# Iterate over all hyperparameter combinations
for hl_config, activation_fn, epochs, lr, batch_size in itertools.product(
    hidden_layer_configs, activation_fns, epoch_counts, learning_rates, batch_sizes):

    print(f"Testing: Hidden Layers={hl_config}, Activation={activation_fn.__name__}, Epochs={epochs}, LR={lr}, Batch={batch_size}")

    # Initialize model, loss function, and optimizer
    model = MLP(input_size, hl_config, activation_fn)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Train the model
    train_losses, test_accuracies = train_model(
        model, criterion, optimizer, X_train, y_train, X_test, y_test, epochs, batch_size
    )

    # Save results for analysis
    results.append({
        'hidden_layers': hl_config,
        'activation_fn': activation_fn.__name__,
        'epochs': epochs,
        'learning_rate': lr,
        'batch_size': batch_size,
        'final_accuracy': test_accuracies[-1]
    })

# Step 5: Save and analyze results
results_df = pd.DataFrame(results)
results_df.to_csv('mlp_hyperparameter_results.csv', index=False)
print("Experiment completed. Results saved to 'mlp_hyperparameter_results.csv'.")


Testing: Hidden Layers=[8], Activation=ReLU, Epochs=10, LR=0.1, Batch=16
Testing: Hidden Layers=[8], Activation=ReLU, Epochs=10, LR=0.1, Batch=32
Testing: Hidden Layers=[8], Activation=ReLU, Epochs=10, LR=0.1, Batch=64
Testing: Hidden Layers=[8], Activation=ReLU, Epochs=10, LR=0.01, Batch=16
Testing: Hidden Layers=[8], Activation=ReLU, Epochs=10, LR=0.01, Batch=32
Testing: Hidden Layers=[8], Activation=ReLU, Epochs=10, LR=0.01, Batch=64
Testing: Hidden Layers=[8], Activation=ReLU, Epochs=10, LR=0.001, Batch=16
Testing: Hidden Layers=[8], Activation=ReLU, Epochs=10, LR=0.001, Batch=32
Testing: Hidden Layers=[8], Activation=ReLU, Epochs=10, LR=0.001, Batch=64
Testing: Hidden Layers=[8], Activation=ReLU, Epochs=25, LR=0.1, Batch=16
Testing: Hidden Layers=[8], Activation=ReLU, Epochs=25, LR=0.1, Batch=32
Testing: Hidden Layers=[8], Activation=ReLU, Epochs=25, LR=0.1, Batch=64
Testing: Hidden Layers=[8], Activation=ReLU, Epochs=25, LR=0.01, Batch=16
Testing: Hidden Layers=[8], Activation=Re